In [15]:
import os

import numpy as np
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import imagesize
import shutil

from joblib import Parallel, delayed

import wandb

In [2]:
api_wandb_key = 'cf1380c2be4499f36827184321c87d7da7942cc7' # tsss do not tell anyone
wandb.login(key=api_wandb_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bucks2309 (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/misha/.netrc


True

In [3]:
FOLD = 0  # which fold to train
DIM = 640
MODEL = "yolov5x"
BATCH = 16
EPOCHS = 15
OPTMIZER = "SGD"

PROJECT = "happywhale-det-public"  # w&b in yolov5
NAME = f"{MODEL}-dim{DIM}-fold{FOLD}"  # w&b for yolov5

REBEL_PATH = '/home/misha/geoframework/'
DATA_PATH = '/media/storage3/data3T/happy_whale/hump_bbox'
ROOT_DIR = os.path.join(REBEL_PATH, 'models/rnd/happy_whale/data/crop')

IMAGE_DIR = os.path.join(DATA_PATH, "train")  # directory to save images
LABEL_DIR = os.path.join(DATA_PATH, "labels")  # directory to save labels

In [4]:
# !mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [5]:
df = pd.read_csv(f"{DATA_PATH}/train.csv")

# Train Data

df["image_id"] = df["Image"]
df["image_path"] = f"{IMAGE_DIR}/" + df.image_id
df["label_path"] = f"{LABEL_DIR}/" + df.image_id.str.replace("jpg", "txt")
df.head(2)

,Image,Id,image_id,image_path,label_path
0,00022e1a.jpg,w_e15442c,00022e1a.jpg,/media/storage3/data3T/happy_whale/hump_bbox/t...,/media/storage3/data3T/happy_whale/hump_bbox/l...
1,000466c4.jpg,w_1287fbc,000466c4.jpg,/media/storage3/data3T/happy_whale/hump_bbox/t...,/media/storage3/data3T/happy_whale/hump_bbox/l...


### Create BBox

In [6]:
from bbox.utils import coco2yolo, coco2voc, voc2yolo, yolo2voc
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [7]:
def point2bbox(points):
    points = np.array(points)
    points = points.astype('int') # str -> int
    points = points.reshape(-1, 2) # shape: (None, ) -> shape: (None, 2) => (x, y) format
    xmin, ymin, xmax, ymax = points[:, 0].min(), points[:, 1].min(), points[:, 0].max(), points[:, 1].max()
    return [[xmin, ymin, xmax, ymax]]

In [8]:
f = open('cropping.txt','rt').read()
id2point = {x.split(',')[0]:x.split(',')[1:] for x in f.split('\n')}
df['point'] = df['image_id'].map(id2point)
df = df[~df.point.isna()]
df['bbox'] = df.point.map(point2bbox)

### Get Image-Size

In [9]:
df = df.progress_apply(get_imgsize, axis=1)
display(df.head(2))

  0%|          | 0/1200 [00:00<?, ?it/s]

,Image,Id,image_id,image_path,label_path,point,bbox,width,height
1,000466c4.jpg,w_1287fbc,000466c4.jpg,/media/storage3/data3T/happy_whale/hump_bbox/t...,/media/storage3/data3T/happy_whale/hump_bbox/l...,"[233, 308, 243, 301, 508, 307, 578, 391, 561, ...","[[233, 301, 578, 434]]",1050,700
2,00087b01.jpg,w_da2efe0,00087b01.jpg,/media/storage3/data3T/happy_whale/hump_bbox/t...,/media/storage3/data3T/happy_whale/hump_bbox/l...,"[1, 63, 80, 8, 1010, 20, 1040, 28, 473, 365, 3...","[[1, 8, 1040, 366]]",1050,368


### Create Labels

In [10]:
# We need to export our labels to YOLO format, with one *.txt file per image (if no objects in image, no *.txt file is required).
# The *.txt file specifications are:

# One row per object

# Each row is class [x_center, y_center, width, height] format.

# Box coordinates must be in normalized xywh format (from 0 - 1).
# If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.

# Class numbers are zero-indexed (start from 0).

# Dataset bbox format is VOC-PASCAL hence [x_min, y_min, x_max, y_max].
# So, we need to convert form VOC-PASCAL to YOLO format.

In [11]:
cnt = 0
all_bboxes = []
bboxes_info = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width = row.width
    bboxes_voc = np.array(row.bbox).astype(np.float32).copy()
    num_bbox = len(bboxes_voc)
    names = ["whale"] * num_bbox
    labels = np.array([0] * num_bbox)[..., None].astype(str)
    ## Create Annotation(YOLO)
    with open(row.label_path, "w") as f:
        if num_bbox < 1:
            annot = ""
            f.write(annot)
            cnt += 1
            continue
        #         bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        bboxes_voc = clip_bbox(bboxes_voc, image_height, image_width)
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        all_bboxes.extend(bboxes_yolo.astype(float))
        bboxes_info.extend([[row.image_id]] * len(bboxes_yolo))
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        string = annot2str(annots)
        f.write(string)
print("Missing:", cnt)

  0%|          | 0/1200 [00:00<?, ?it/s]

Missing: 0


### Create Folds
Number of samples aren't same in each fold which can create large variance in Cross-Validation.

In [16]:
kf = KFold(n_splits=6, random_state=42, shuffle=True)
df = df.reset_index(drop=True)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    df.loc[val_idx, 'fold'] = fold
df.fold.value_counts()

2    200
5    200
1    200
4    200
0    200
3    200
Name: fold, dtype: int64

### BBox Distribution

In [18]:
bbox_df = pd.DataFrame(np.concatenate([bboxes_info, all_bboxes], axis=1),
             columns=['image_id','xmid','ymid','w','h'])
bbox_df[['xmid','ymid','w','h']] = bbox_df[['xmid','ymid','w','h']].astype(float)
bbox_df['area'] = bbox_df.w * bbox_df.h
bbox_df = bbox_df.merge(df[['image_id','fold']], on='image_id', how='left')
bbox_df.head(2)

,image_id,xmid,ymid,w,h,area,fold
0,000466c4.jpg,0.386190,0.525000,0.328571,0.190000,0.062429,2
1,00087b01.jpg,0.495714,0.508152,0.989524,0.972826,0.962635,5
